# Document Scanner Using OpenCV

### This notebook demonstrates how to scan a document from an image using OpenCV. The process includes reading the image, preprocessing it, detecting the document's contours, and applying a perspective transformation to obtain a scanned effect. 

#### We will import the necessary libraries: 
1. OpenCV for image processing 
2. NumPy for numerical operations 
3. Matplotlib for displaying images.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### Defining Helper Functions

1. **order_points(pts)**: This function orders the points of the detected contour in a specific order (top-left, top-right, bottom-right, bottom-left).
2. **four_point_transform(image, pts)**: This function applies a perspective transformation to the image based on the ordered points.


In [2]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    return warped

### **scan_document(image_path)**: This function orchestrates the entire scanning process from reading the image to returning the scanned output.
1. Image Preprocessing: The code converts the image to grayscale and applies Gaussian blur to reduce noise. Then it uses the Canny edge detector algorithm to find edges - this is a mathematical algorithm, not a trained model.
2. Contour Detection: The code finds contours in the edge-detected image using geometric algorithms to trace the boundaries.
3. Document Border Detection: it approximates the contours and find the document's four corners
4. Perspective Transform: Finally, it applies a perspective transform using matrix operations to get a top-down view.

In [3]:
def scan_document(image_path):
    image = cv2.imread(image_path)
    orig = image.copy()
    ratio = image.shape[0] / 500.0
    image = cv2.resize(image, (int(image.shape[1] / ratio), 500))

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)

    kernel = np.ones((3, 3), np.uint8)
    edged = cv2.dilate(edged, kernel, iterations=1)
    edged = cv2.erode(edged, kernel, iterations=1)


    cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

    screenCnt = None
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        if len(approx) == 4:
            screenCnt = approx
            break

    if screenCnt is None:
        print("No document detected. Please try another image.")
        return

    warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    T = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return image, T

## Scanning a Document

In this cell we will specify the path to the input image and call the `scan_document` function to perform the scanning operation.


In [ ]:
if __name__ == "__main__":
    input_image_path = "sample_images/dollar_bill.JPG"  # Replace this with the path of your document image 
    orig_resized, scanned_image = scan_document(input_image_path)

    if scanned_image is not None:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))

        # Show the original image
        ax1.imshow(cv2.cvtColor(orig_resized, cv2.COLOR_BGR2RGB))
        ax1.axis('off')
        ax1.set_title('Original Image')

        # Show the scanned image
        ax2.imshow(scanned_image, cmap='gray')
        ax2.axis('off')
        ax2.set_title('Scanned Image')

        plt.show()